In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Data Loading and Cleaning
def load_and_clean_data(file_path):
    df = pd.read_csv(file_path)
    
    # Clean column names
    df.columns = df.columns.str.strip().str.replace('\n', '')
    
    # keep both buys and sells (not just buys)
    # df = df[df.transaction_type.isin(['Buy', 'Sell'])]
    
    # Convert date columns to datetime
    date_cols = ['DATE OF ALLOTMENT/ACQUISITION FROM', 'DATE OF ALLOTMENT/ACQUISITION TO']
    for col in date_cols:
        df[col] = pd.to_datetime(df[col], format='%d-%b-%Y', errors='coerce')
    
    # Clean numeric columns
    df['NO. OF SECURITIES (ACQUIRED/DISPLOSED)'] = pd.to_numeric(
        df['NO. OF SECURITIES (ACQUIRED/DISPLOSED)'], errors='coerce'
    )
    
    # Clean VALUE column
    df['VALUE OF SECURITY (ACQUIRED/DISPLOSED)'] = df['VALUE OF SECURITY (ACQUIRED/DISPLOSED)'].replace('-', np.nan)
    df['VALUE OF SECURITY (ACQUIRED/DISPLOSED)'] = pd.to_numeric(
        df['VALUE OF SECURITY (ACQUIRED/DISPLOSED)'], errors='coerce'
    )
    
    return df

def filter_relevant_data(df):
    # Filter out rows with missing critical information
    df_filtered = df.dropna(subset=[
        'SYMBOL', 'DATE OF ALLOTMENT/ACQUISITION FROM', 
        'NO. OF SECURITIES (ACQUIRED/DISPLOSED)',
        'ACQUISITION/DISPOSAL TRANSACTION TYPE'
    ])
    
    # Filter for only Buy and Sell transactions
    df_filtered = df_filtered[df_filtered['ACQUISITION/DISPOSAL TRANSACTION TYPE'].isin(['Buy', 'Sell'])]
    
    # Filter for market transactions only
    market_modes = ['Market Sale', 'Market Purchase', 'Off Market']
    df_filtered = df_filtered[df_filtered['MODE OF ACQUISITION'].isin(market_modes)]
    
    # Filter for significant insider categories
    significant_categories = ['Promoters', 'Promoter Group', 'Director', 'Key Managerial Personnel']
    df_filtered = df_filtered[df_filtered['CATEGORY OF PERSON'].isin(significant_categories)]
    
    # Create transaction_type column (1 for Buy, -1 for Sell)
    df_filtered['transaction_type'] = df_filtered['ACQUISITION/DISPOSAL TRANSACTION TYPE'].map({
        'Buy': 1, 'Sell': -1
    })
    
    # Filter for reasonable date range (2022-2025)
    start_date = pd.to_datetime('2022-01-01')
    end_date = pd.to_datetime('2025-12-31')
    df_filtered = df_filtered[
        (df_filtered['DATE OF ALLOTMENT/ACQUISITION FROM'] >= start_date) & 
        (df_filtered['DATE OF ALLOTMENT/ACQUISITION FROM'] <= end_date)
    ]
    
    return df_filtered

# Feature Engineering
def create_insider_weights():
    
    return {
        'Promoters': 4,
        'Promoter Group': 3,
        'Director': 2,
        'Key Managerial Personnel': 1
    }

def add_insider_weights(df):
    weights = create_insider_weights()
    df['insider_weight'] = df['CATEGORY OF PERSON'].map(weights)
    return df

def calculate_transaction_metrics(df):
    # Calculate transaction value (if available)
    df['transaction_value'] = df['VALUE OF SECURITY (ACQUIRED/DISPLOSED)'].fillna(0)
    
    # Calculate weighted transaction score
    df['weighted_score'] = df['transaction_type'] * df['insider_weight'] * np.log1p(df['NO. OF SECURITIES (ACQUIRED/DISPLOSED)'])
    
    # Add transaction date as reference
    df['transaction_date'] = df['DATE OF ALLOTMENT/ACQUISITION FROM']
    
    return df

# Consensus Signal Generation
class ConsensusParameters:
    def __init__(self, lookback_window=3, min_insiders=3, min_net_score=2.0, 
                 signal_hold_period=30, min_transaction_value=1000000):
        self.lookback_window = lookback_window  # days
        self.min_insiders = min_insiders        # minimum number of insiders needed
        self.min_net_score = min_net_score      # minimum weighted net score
        self.signal_hold_period = signal_hold_period  # days to hold signal
        self.min_transaction_value = min_transaction_value  # minimum transaction value

def generate_consensus_signals(df, params):
    signals = []
    
    # Group by company symbol
    for symbol in df['SYMBOL'].unique():
        company_data = df[df['SYMBOL'] == symbol].copy()
        company_data = company_data.sort_values('transaction_date')
        
        # Create rolling window analysis
        signals.extend(analyze_company_consensus(company_data, params))
    
    signals_df = pd.DataFrame(signals)
    # Ensure signal_date is datetime and drop invalid
    signals_df['signal_date'] = pd.to_datetime(signals_df['signal_date'], errors='coerce')
    signals_df = signals_df.dropna(subset=['signal_date'])
    return signals_df

def analyze_company_consensus(company_data, params):
    signals = []
    
    # Ensure transaction_date is datetime
    company_data['transaction_date'] = pd.to_datetime(company_data['transaction_date'], errors='coerce')
    company_data = company_data.dropna(subset=['transaction_date'])
    
    # Get unique transaction dates
    transaction_dates = company_data['transaction_date'].unique()
    
    for date in transaction_dates:
        # Define lookback window
        window_start = date - timedelta(days=params.lookback_window)
        window_end = date
        
        # Filter data within window
        window_data = company_data[
            (company_data['transaction_date'] >= window_start) & 
            (company_data['transaction_date'] <= window_end)
        ]
        
        # Calculate consensus metrics
        signal = calculate_consensus_signal(window_data, date, params)
        if signal:
            signals.append(signal)
    
    return signals

def calculate_consensus_signal(window_data, signal_date, params):
    if len(window_data) < params.min_insiders:
        return None
    
    # Count unique insiders
    unique_insiders = window_data['NAME OF THE ACQUIRER/DISPOSER'].nunique()
    if unique_insiders < params.min_insiders:
        return None
    
    # Calculate net weighted score
    net_score = window_data['weighted_score'].sum()
    
    # Calculate transaction direction consensus
    buy_score = window_data[window_data['transaction_type'] == 1]['weighted_score'].sum()
    sell_score = abs(window_data[window_data['transaction_type'] == -1]['weighted_score'].sum())
    
    # Determine signal direction
    if buy_score > sell_score and net_score > params.min_net_score:
        signal_direction = 'BUY'
    elif sell_score > buy_score and abs(net_score) > params.min_net_score:
        signal_direction = 'SELL'
    else:
        return None
    
    # Calculate signal strength
    signal_strength = abs(net_score) / params.min_net_score
    
    return {
        'symbol': window_data['SYMBOL'].iloc[0],
        'company': window_data['COMPANY'].iloc[0],
        'signal_date': signal_date,
        'signal_direction': signal_direction,
        'signal_strength': signal_strength,
        'net_score': net_score,
        'unique_insiders': unique_insiders,
        'total_transactions': len(window_data),
        'buy_score': buy_score,
        'sell_score': sell_score
    }

# Signal Validation and Filtering
def filter_overlapping_signals(signals_df, params):
    filtered_signals = []
    
    for symbol in signals_df['symbol'].unique():
        company_signals = signals_df[signals_df['symbol'] == symbol].copy()
        company_signals = company_signals.sort_values('signal_date')
        
        last_signal_date = None
        for _, signal in company_signals.iterrows():
            if last_signal_date is None or (signal['signal_date'] - last_signal_date).days > params.signal_hold_period:
                filtered_signals.append(signal)
                last_signal_date = signal['signal_date']
    
    return pd.DataFrame(filtered_signals)

def add_quality_score(signals_df):
    # Normalize signal strength
    max_strength = signals_df['signal_strength'].max()
    signals_df['normalized_strength'] = signals_df['signal_strength'] / max_strength
    
    # Calculate quality score based on multiple factors
    signals_df['quality_score'] = (
        0.4 * signals_df['normalized_strength'] +
        0.3 * (signals_df['unique_insiders'] / signals_df['unique_insiders'].max()) +
        0.3 * (signals_df['total_transactions'] / signals_df['total_transactions'].max())
    )
    
    return signals_df

# Portfolio Construction
def calculate_position_sizes(signals_df, total_capital=1000000):
    # Filter for high quality signals
    high_quality_signals = signals_df[signals_df['quality_score'] >= 0.6]
    
    # Calculate position sizes
    total_quality_score = high_quality_signals['quality_score'].sum()
    high_quality_signals['position_size'] = (
        high_quality_signals['quality_score'] / total_quality_score * total_capital
    )
    
    return high_quality_signals

def apply_diversification_rules(portfolio_df, max_position_pct=0.15):
    total_capital = portfolio_df['position_size'].sum()
    max_position_size = total_capital * max_position_pct
    
    # Cap individual positions
    portfolio_df['position_size'] = portfolio_df['position_size'].clip(upper=max_position_size)
    
    # Rebalance to maintain total capital
    scale_factor = total_capital / portfolio_df['position_size'].sum()
    portfolio_df['position_size'] *= scale_factor
    
    return portfolio_df

# Backtesting Framework
def load_price_data(symbols):
    # This would typically connect to a financial data provider
    # For demonstration, we'll create a placeholder
    price_data = {}
    for symbol in symbols:
        # Placeholder for actual price data loading
        price_data[symbol] = generate_mock_price_data(symbol)
    return price_data

def generate_mock_price_data(symbol):
    dates = pd.date_range('2022-01-01', '2025-07-15', freq='D')
    prices = 100 * np.cumprod(1 + np.random.normal(0, 0.02, len(dates)))
    return pd.DataFrame({'date': dates, 'price': prices})

def run_backtest(portfolio_df, price_data, params):
    results = []
    
    for _, position in portfolio_df.iterrows():
        symbol = position['symbol']
        entry_date = position['signal_date']
        exit_date = entry_date + timedelta(days=params.signal_hold_period)
        
        # Get price data for this symbol
        symbol_prices = price_data.get(symbol)
        if symbol_prices is None:
            continue
        
        # Calculate returns
        entry_price = get_price_on_date(symbol_prices, entry_date)
        exit_price = get_price_on_date(symbol_prices, exit_date)
        
        if entry_price and exit_price:
            if position['signal_direction'] == 'BUY':
                return_pct = (exit_price - entry_price) / entry_price
            else:  # SELL
                return_pct = (entry_price - exit_price) / entry_price
            
            results.append({
                'symbol': symbol,
                'entry_date': entry_date,
                'exit_date': exit_date,
                'entry_price': entry_price,
                'exit_price': exit_price,
                'return_pct': return_pct,
                'position_size': position['position_size'],
                'pnl': return_pct * position['position_size']
            })
    
    return pd.DataFrame(results)

def get_price_on_date(price_data, target_date):
    # … [earlier integer‐date handling omitted for brevity] …

    # Ensure target_date is a pd.Timestamp
    target_date = pd.to_datetime(target_date, errors='coerce')
    if pd.isna(target_date):
        return None

    # Sort and re‐index by date so .asof() compares Timestamps
    price_series = (
        price_data
        .sort_values('date')
        .set_index('date')['price']
    )
    
    # asof now works on the datetime index
    closest_price = price_series.asof(target_date)
    if pd.isna(closest_price):
        return None
    
    return closest_price


# Performance Analysis
def calculate_performance_metrics(backtest_results):
    if len(backtest_results) == 0:
        return {}
    
    returns = backtest_results['return_pct']
    pnl = backtest_results['pnl']
    
    metrics = {
        'total_trades': len(backtest_results),
        'total_return': pnl.sum(),
        'win_rate': (returns > 0).mean(),
        'avg_return': returns.mean(),
        'volatility': returns.std(),
        'sharpe_ratio': returns.mean() / returns.std() if returns.std() > 0 else 0,
        'max_drawdown': calculate_max_drawdown(pnl),
        'profit_factor': calculate_profit_factor(pnl),
        'avg_win': returns[returns > 0].mean() if (returns > 0).any() else 0,
        'avg_loss': returns[returns < 0].mean() if (returns < 0).any() else 0
    }
    
    return metrics

def calculate_max_drawdown(pnl_series):
    cumulative_pnl = pnl_series.cumsum()
    rolling_max = cumulative_pnl.expanding().max()
    drawdown = cumulative_pnl - rolling_max
    return drawdown.min()

def calculate_profit_factor(pnl_series):
    gross_profit = pnl_series[pnl_series > 0].sum()
    gross_loss = abs(pnl_series[pnl_series < 0].sum())
    return gross_profit / gross_loss if gross_loss > 0 else np.inf

# Main Strategy Class
class ConsensusInsiderStrategy:
    def __init__(self, params=None):
        self.params = params or ConsensusParameters()
        self.raw_data = None
        self.cleaned_data = None
        self.signals = None
        self.portfolio = None
        self.backtest_results = None
        self.performance_metrics = None
    
    def load_data(self, file_path):
        self.raw_data = load_and_clean_data(file_path)
        self.cleaned_data = self.preprocess_data()
        return self
    
    def preprocess_data(self):
        df = filter_relevant_data(self.raw_data)
        df = add_insider_weights(df)
        df = calculate_transaction_metrics(df)
        return df
    
    def generate_signals(self):
        self.signals = generate_consensus_signals(self.cleaned_data, self.params)
        self.signals = filter_overlapping_signals(self.signals, self.params)
        self.signals = add_quality_score(self.signals)
        return self
    
    def build_portfolio(self, total_capital=1000000):
        self.portfolio = calculate_position_sizes(self.signals, total_capital)
        self.portfolio = apply_diversification_rules(self.portfolio)
        return self
    
    def run_backtest(self, price_data=None):
        if price_data is None:
            symbols = self.portfolio['symbol'].unique()
            price_data = load_price_data(symbols)
        
        self.backtest_results = run_backtest(self.portfolio, price_data, self.params)
        self.performance_metrics = calculate_performance_metrics(self.backtest_results)
        return self
    
    def get_results(self):
        return {
            'signals': self.signals,
            'portfolio': self.portfolio,
            'backtest_results': self.backtest_results,
            'performance_metrics': self.performance_metrics
        }

# Parameter Sweep Function
def run_parameter_sweep(param_sets, file_path='CF-Insider-Trading-equities-14-07-2022-to-14-07-2025.csv', 
                        total_capital=1000000):
    
    sweep_results = {}
    
    for i, params_dict in enumerate(param_sets):
        # Create parameters object with custom values
        params = ConsensusParameters(**params_dict)
        
        # Initialize and run strategy
        strategy = ConsensusInsiderStrategy(params=params)
        strategy.load_data(file_path)
        strategy.generate_signals()
        strategy.build_portfolio(total_capital=total_capital)
        strategy.run_backtest()
        
        # Get and store results
        results = strategy.get_results()
        sweep_results[f'Set_{i+1}'] = {
            'parameters': params_dict,
            'performance_metrics': results['performance_metrics']
        }
        
        # Print summary for this set
        print(f"\nResults for Parameter Set {i+1}: {params_dict}")
        for metric, value in results['performance_metrics'].items():
            print(f"{metric}: {value:.4f}")
    
    return sweep_results


In [ ]:
import itertools

lookback_windows       = [3, 7]
min_insiders_list      = [1, 2]
min_net_scores         = [0.02, 0.08, 0.16, 0.2]
signal_hold_periods    = [1, 5, 10]
min_transaction_values = [0, 1000, 50000]

all_combos = list(itertools.product(
    lookback_windows,
    min_insiders_list,
    min_net_scores,
    signal_hold_periods,
    min_transaction_values
))

param_sets = []
for (lw, mi, mns, shp, mtv) in all_combos[:]:
    param_sets.append({
        'lookback_window'      : lw,
        'min_insiders'         : mi,
        'min_net_score'        : mns,
        'signal_hold_period'   : shp,
        'min_transaction_value': mtv
    })

for i, ps in enumerate(param_sets, 1):
    print(f"Set {i:2d}:", ps)

sweep_results = run_parameter_sweep(param_sets)

Set  1: {'lookback_window': 3, 'min_insiders': 1, 'min_net_score': 0.02, 'signal_hold_period': 1, 'min_transaction_value': 0}
Set  2: {'lookback_window': 3, 'min_insiders': 1, 'min_net_score': 0.02, 'signal_hold_period': 1, 'min_transaction_value': 1000}
Set  3: {'lookback_window': 3, 'min_insiders': 1, 'min_net_score': 0.02, 'signal_hold_period': 1, 'min_transaction_value': 50000}
Set  4: {'lookback_window': 3, 'min_insiders': 1, 'min_net_score': 0.02, 'signal_hold_period': 5, 'min_transaction_value': 0}
Set  5: {'lookback_window': 3, 'min_insiders': 1, 'min_net_score': 0.02, 'signal_hold_period': 5, 'min_transaction_value': 1000}
Set  6: {'lookback_window': 3, 'min_insiders': 1, 'min_net_score': 0.02, 'signal_hold_period': 5, 'min_transaction_value': 50000}
Set  7: {'lookback_window': 3, 'min_insiders': 1, 'min_net_score': 0.02, 'signal_hold_period': 10, 'min_transaction_value': 0}
Set  8: {'lookback_window': 3, 'min_insiders': 1, 'min_net_score': 0.02, 'signal_hold_period': 10, 'min

In [ ]:
import pandas as pd

try:
    from tabulate import tabulate

    def pretty_print(df: pd.DataFrame, title: str):
        print(f"\n=== {title} ===")
        print(tabulate(df, headers='keys', tablefmt='psql', floatfmt=".4f"))
except ImportError:
    def pretty_print(df: pd.DataFrame, title: str):
        print(f"\n=== {title} ===")
        print(df.to_string(float_format=lambda x: f"{x:,.4f}"))

def rank_strategies(sweep_results, total_capital=1000000):
    records = []
    for name, res in sweep_results.items():
        m = res['performance_metrics']
        records.append({'set': name, **m})
    df = pd.DataFrame(records).set_index('set')

    pretty_print(df, "ALL STRATEGIES")

    df_filtered = df[
        (df['profit_factor'] > 1.0) &
        (df['max_drawdown'] > -0.15 * total_capital) &
        (df['total_trades'] >= 10)
    ]
    pretty_print(df_filtered, "AFTER FILTERS")

    if df_filtered.empty:
        print("\nNo strategies passed filters; ranking all of them instead.")
        df_filtered = df.copy()

    df_filtered['sharpe_rank'] = df_filtered['sharpe_ratio'].rank(pct=True)
    df_filtered['pf_rank']     = df_filtered['profit_factor'].rank(pct=True)
    df_filtered['dd_rank']     = 1 - df_filtered['max_drawdown'].abs().rank(pct=True)

    w1, w2, w3 = 0.5, 0.3, 0.2
    df_filtered['composite_score'] = (
        w1 * df_filtered['sharpe_rank']
      + w2 * df_filtered['pf_rank']
      + w3 * df_filtered['dd_rank']
    )

    ranked = df_filtered.sort_values('composite_score', ascending=False)
    pretty_print(ranked[['composite_score', 'total_trades','sharpe_ratio', 'profit_factor', 'max_drawdown']], 
                 "RANKED STRATEGIES")

    return ranked

In [ ]:
ranked_df = rank_strategies(sweep_results)


=== ALL STRATEGIES ===
         total_trades  total_return  win_rate  avg_return  volatility  sharpe_ratio  max_drawdown  profit_factor  avg_win  avg_loss
set                                                                                                                                
Set_1               9    3,662.4765    0.5556      0.0031      0.0188        0.1674   -2,671.4803         1.6617   0.0158   -0.0127
Set_2               9   -3,924.1278    0.4444     -0.0040      0.0220       -0.1834  -10,461.4683         0.6249   0.0140   -0.0185
Set_3               9  -12,233.7772    0.3333     -0.0110      0.0232       -0.4765  -10,951.0777         0.2888   0.0158   -0.0245
Set_4               5   -6,219.7179    0.4000     -0.0062      0.0314       -0.1979  -11,161.4087         0.5436   0.0185   -0.0227
Set_5               5      721.7559    0.6000      0.0007      0.0619        0.0117  -18,251.6623         1.0327   0.0380   -0.0553
Set_6               5  -11,149.9400    0.4000     -0

In [ ]:
ranked_df[['composite_score', 'total_trades','sharpe_ratio', 'profit_factor', 'max_drawdown']].head()

,composite_score,total_trades,sharpe_ratio,profit_factor,max_drawdown
set,,,,,
Set_24,0.991667,5,2.420938,inf,0.000000
Set_68,0.988194,5,2.288251,inf,0.000000
Set_133,0.984722,5,2.008714,inf,0.000000
Set_52,0.970833,5,1.320488,197.151107,-284.903258
Set_142,0.961111,5,1.003541,11.259567,0.000000
